# Use pyquery-ql.py

Send a graphql query to GitHub
and pretty print output.

Supports Python 3.6+

In [1]:
import json
import os
import pprint

import requests

In [2]:
# get api token and set authorization
api_token = os.environ['GITHUB_API_TOKEN']
headers = {'Authorization': f'token {api_token}'}

In [3]:
# set url to a graphql endpoint
url = 'https://api.github.com/graphql'

In [4]:
# add a json query
query = """
{
  organization(login: "jupyterhub") {
    repositories(first: 30) {
      totalCount
      edges {
        node {
          name
          url
          issues(states: OPEN) {
            totalCount
          }
          pullRequests(states: OPEN) {
            totalCount
          }
        }
      }
    }
  }
}
"""

In [5]:
# submit the request
r = requests.post(url=url, json={'query': query}, headers=headers)

In [28]:
r.text

'{"data":{"organization":{"repositories":{"totalCount":27,"edges":[{"node":{"name":"jupyterhub","url":"https://github.com/jupyterhub/jupyterhub","issues":{"totalCount":143},"pullRequests":{"totalCount":6}}},{"node":{"name":"configurable-http-proxy","url":"https://github.com/jupyterhub/configurable-http-proxy","issues":{"totalCount":10},"pullRequests":{"totalCount":0}}},{"node":{"name":"oauthenticator","url":"https://github.com/jupyterhub/oauthenticator","issues":{"totalCount":13},"pullRequests":{"totalCount":2}}},{"node":{"name":"dockerspawner","url":"https://github.com/jupyterhub/dockerspawner","issues":{"totalCount":22},"pullRequests":{"totalCount":1}}},{"node":{"name":"sudospawner","url":"https://github.com/jupyterhub/sudospawner","issues":{"totalCount":2},"pullRequests":{"totalCount":0}}},{"node":{"name":"batchspawner","url":"https://github.com/jupyterhub/batchspawner","issues":{"totalCount":12},"pullRequests":{"totalCount":5}}},{"node":{"name":"kubespawner","url":"https://github.c

In [27]:
data = json.loads(r.text)

{'data': {'organization': {'repositories': {'edges': [{'node': {'issues': {'totalCount': 143},
       'name': 'jupyterhub',
       'pullRequests': {'totalCount': 6},
       'url': 'https://github.com/jupyterhub/jupyterhub'}},
     {'node': {'issues': {'totalCount': 10},
       'name': 'configurable-http-proxy',
       'pullRequests': {'totalCount': 0},
       'url': 'https://github.com/jupyterhub/configurable-http-proxy'}},
     {'node': {'issues': {'totalCount': 13},
       'name': 'oauthenticator',
       'pullRequests': {'totalCount': 2},
       'url': 'https://github.com/jupyterhub/oauthenticator'}},
     {'node': {'issues': {'totalCount': 22},
       'name': 'dockerspawner',
       'pullRequests': {'totalCount': 1},
       'url': 'https://github.com/jupyterhub/dockerspawner'}},
     {'node': {'issues': {'totalCount': 2},
       'name': 'sudospawner',
       'pullRequests': {'totalCount': 0},
       'url': 'https://github.com/jupyterhub/sudospawner'}},
     {'node': {'issues': {'to

In [1]:
import requests
import json
from itertools import zip_longest
import pandas as pd
import numpy as np

In [2]:
#This is a little helper funciton that will take x of something, and let you chunk through a subset
#Note that 'n' is the chunk size
def _grouper(n, iterable, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

In [22]:
#Since you want a report by month, lets just create a function to get all the chat IDs for a two digit month and 4 digit year

def get_ids(data):
    all_ids = []
    response = json.loads(data)
    for id in response['data']:
        all_ids.append('id')
    return all_ids

In [62]:
all_ids = []
for id in data['data']:
    print(id)
    all_ids.append(id)
    
    for id in data['data'][all_ids[0]]:
        print(id)
        all_ids.append(id)
        print(all_ids)
        
    for id in data['data'][all_ids[0]][all_ids[1]]:
        print(id)
        all_ids.append(id)
    
    print(all_ids)
    print(data['data']['edges'])


organization
repositories
['organization', 'repositories']
totalCount
edges
['organization', 'repositories', 'totalCount', 'edges']


KeyError: 'edges'

In [4]:
def build_csv_data(queried_id_list):
  all_chats_for_csv = []
  for chunk in _grouper(50, queried_id_list):
    filtered = [id for id in chunk if id is not None] #Only needed for last set of 50 that will have some sort of remainder filled with 'None' values by the grouper() function
    url = "{0}/chats".format(config['base_path'])
    stringified_ids = ",".join(filtered)
    params = {"ids":stringified_ids}
    r = s.get(url, params=params)
    if r.status_code == 200:
      response = json.loads(r.text)
      docs = response['docs']
      for k in docs:
        doc_obj = docs[k] #This is the 'doc' object. Note that some keys may not exist for all items. Duration does not exist for offline messages for example
        all_chats_for_csv.append(doc_obj)
    else:
      print("error getting bulk chats")
  return all_chats_for_csv

In [7]:
#grab the ids for the specified range NOTE!! This is where you can change the year/month for the date range.
queried_id_list = get_chat_ids_for_month(2016, 10)

#turn those into raw objects
csv_obj_data = build_csv_data(queried_id_list)

#create a list to hold our rows
all_rows_as_obj = []

#turn the raw objects into nice and tasty ones that pandas can digest
for record in csv_obj_data:
  try:
    response_time = record.get('response_time', {})
    count = record.get('count',{})
    csv_obj = {
      "id": record['id'],
      "agent": ",".join(record.get('agent_names', "N/A")),
      "visitor": record['visitor']['name'],
      "department": record['department_name'],
      "url": record.get('webpath', "N/A"),
      "missed": record.get('missed', "N/A"),
      "resp_first": response_time.get('first', []),
      "resp_max": response_time.get('max', []),
      "resp_avg": response_time.get('avg', []),
      "start timestamp": record['session']['start_date'],
      "end timestamp": record['session']['end_date'],
      "total messages": count.get('total', 0),
      "Agent Msg Count": count.get('agent', 0),
      "Visitor Msg Count": count.get('visitor', 0),
      "rating": record.get('rating',[]),
      "ticket_id": record['zendesk_ticket_id']
    }
  except KeyError as e:
    print(e)
    print("key error occured for record with id: {}".format(record['id']))
    print(record)
    pass
  all_rows_as_obj.append(csv_obj)

#with prettier objects our column names can just come from a call to the first object's keys
col_keys = all_rows_as_obj[0].keys()

#create the dataframe, list comprehension creates an array of rows based on the column names we got
#Let pandas do the hard part with csvs
csv_frame = pd.DataFrame([[i[j] for j in col_keys] for i in all_rows_as_obj], columns=col_keys)

#coerce ticket_id to be int not float
csv_frame['ticket_id'] = csv_frame['ticket_id'].fillna(0).astype(np.int64)

#fill blank values with the string 'N/A'
csv_frame = csv_frame.fillna("N/A")

#output to a file
csv_frame.to_csv('october_stats.csv')

Walk the JSON response contents

In [8]:
issue_data = data['data']

In [9]:
org = issue_data['organization']

In [10]:
repos = org['repositories']

In [11]:
edges = repos['edges']

In [12]:
# edges[0]['node']

In [13]:
# print(edges[0]['node']['name'] + '---' + edges[0]['node']['url'])

In [14]:
# for edge in edges:
#    pprint.pprint(edge)

Bring into pandas

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame.from_records(edges)

In [17]:
df.columns

Index(['node'], dtype='object')

In [18]:
# df['node']

Generate basic report of total open issues

In [19]:
print(f"{'Repo':30} {'OpenIssues':11} {'OpenPRs':9} {'URL'}")

line = 26
while line > 0:
    lineout = f"{df['node'][line]['name']:30} {(df['node'][line]['issues']['totalCount']):8} {(df['node'][line]['pullRequests']['totalCount']):8}     {df['node'][line]['url']}"
    print(lineout)
    line -= 1

Repo                           OpenIssues  OpenPRs   URL
team-compass                          2        1     https://github.com/jupyterhub/team-compass
nullauthenticator                     0        0     https://github.com/jupyterhub/nullauthenticator
binder                               10        1     https://github.com/jupyterhub/binder
mybinder.org-deploy                  24        1     https://github.com/jupyterhub/mybinder.org-deploy
binderhub                            77        6     https://github.com/jupyterhub/binderhub
helm-chart                            3        0     https://github.com/jupyterhub/helm-chart
zero-to-jupyterhub-k8s               89        3     https://github.com/jupyterhub/zero-to-jupyterhub-k8s
tmpauthenticator                      0        0     https://github.com/jupyterhub/tmpauthenticator
nbrsessionproxy                       4        2     https://github.com/jupyterhub/nbrsessionproxy
hubshare                             10        0     https://

In [20]:
df.dtypes

node    object
dtype: object

In [21]:
df.head()

,node
0,"{'name': 'jupyterhub', 'url': 'https://github...."
1,"{'name': 'configurable-http-proxy', 'url': 'ht..."
2,"{'name': 'oauthenticator', 'url': 'https://git..."
3,"{'name': 'dockerspawner', 'url': 'https://gith..."
4,"{'name': 'sudospawner', 'url': 'https://github..."


In [22]:
df.index

RangeIndex(start=0, stop=27, step=1)

In [23]:
df.columns

Index(['node'], dtype='object')

In [24]:
df.values

array([[ {'name': 'jupyterhub', 'url': 'https://github.com/jupyterhub/jupyterhub', 'issues': {'totalCount': 143}, 'pullRequests': {'totalCount': 6}}],
       [ {'name': 'configurable-http-proxy', 'url': 'https://github.com/jupyterhub/configurable-http-proxy', 'issues': {'totalCount': 10}, 'pullRequests': {'totalCount': 0}}],
       [ {'name': 'oauthenticator', 'url': 'https://github.com/jupyterhub/oauthenticator', 'issues': {'totalCount': 13}, 'pullRequests': {'totalCount': 2}}],
       [ {'name': 'dockerspawner', 'url': 'https://github.com/jupyterhub/dockerspawner', 'issues': {'totalCount': 22}, 'pullRequests': {'totalCount': 1}}],
       [ {'name': 'sudospawner', 'url': 'https://github.com/jupyterhub/sudospawner', 'issues': {'totalCount': 2}, 'pullRequests': {'totalCount': 0}}],
       [ {'name': 'batchspawner', 'url': 'https://github.com/jupyterhub/batchspawner', 'issues': {'totalCount': 12}, 'pullRequests': {'totalCount': 5}}],
       [ {'name': 'kubespawner', 'url': 'https://githu

In [25]:
df.sort_index

<bound method DataFrame.sort_index of                                                  node
0   {'name': 'jupyterhub', 'url': 'https://github....
1   {'name': 'configurable-http-proxy', 'url': 'ht...
2   {'name': 'oauthenticator', 'url': 'https://git...
3   {'name': 'dockerspawner', 'url': 'https://gith...
4   {'name': 'sudospawner', 'url': 'https://github...
5   {'name': 'batchspawner', 'url': 'https://githu...
6   {'name': 'kubespawner', 'url': 'https://github...
7   {'name': 'ldapauthenticator', 'url': 'https://...
8   {'name': 'jupyterhub-deploy-docker', 'url': 'h...
9   {'name': 'jupyterhub-deploy-teaching', 'url': ...
10  {'name': 'jupyterhub-tutorial', 'url': 'https:...
11  {'name': 'jupyterhub-deploy-hpc', 'url': 'http...
12  {'name': 'systemdspawner', 'url': 'https://git...
13  {'name': 'wrapspawner', 'url': 'https://github...
14  {'name': 'jupyterlab-hub', 'url': 'https://git...
15  {'name': 'nbserverproxy', 'url': 'https://gith...
16  {'name': 'jupyterhub-example-kerberos', 

In [26]:
# output data to a csv
# df.to_csv('issue_data.csv')